In [138]:
import numpy as np
import pandas as pd
#import user data
df_user = pd.read_csv('ratings.csv')
print(df_user)

        userId  movieId  rating   timestamp
0            1       31     2.5  1260759144
1            1     1029     3.0  1260759179
2            1     1061     3.0  1260759182
3            1     1129     2.0  1260759185
4            1     1172     4.0  1260759205
...        ...      ...     ...         ...
99999      671     6268     2.5  1065579370
100000     671     6269     4.0  1065149201
100001     671     6365     4.0  1070940363
100002     671     6385     2.5  1070979663
100003     671     6565     3.5  1074784724

[100004 rows x 4 columns]


In [139]:
#checking for null_values
null_values = df_user.isnull().sum()
#count null values for eachn column
print(null_values)

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64


In [140]:
# Creating dictionary containing each user and a list of the movies they have rated and their corresponding rating
user_movie_dict = df_user.groupby('userId').apply(lambda x: list(zip(x['movieId'], x['rating']))).to_dict()


#reduce amount of users to prevent overload in system

items_list = list(user_movie_dict.items())


# Slice the list
sliced_items = items_list

# Convert back to dictionary
sliced_user_dict = dict(sliced_items)



In [141]:
#import umovie data
df_movie = pd.read_csv('movies.csv')
print(df_movie)

      movieId                                              title  \
0           1                                   Toy Story (1995)   
1           2                                     Jumanji (1995)   
2           3                            Grumpier Old Men (1995)   
3           4                           Waiting to Exhale (1995)   
4           5                 Father of the Bride Part II (1995)   
...       ...                                                ...   
9120   162672                                Mohenjo Daro (2016)   
9121   163056                               Shin Godzilla (2016)   
9122   163949  The Beatles: Eight Days a Week - The Touring Y...   
9123   164977                           The Gay Desperado (1936)   
9124   164979                              Women of '69, Unboxed   

                                           genres  
0     Adventure|Animation|Children|Comedy|Fantasy  
1                      Adventure|Children|Fantasy  
2                          

In [142]:
#checking for null_values
null_values = df_movie.isnull().sum()
#count null values for eachn column
print(null_values)

movieId    0
title      0
genres     0
dtype: int64


In [144]:
#listing number of movies
movie_id_list = df_movie['movieId'].tolist()
movie_id_list = movie_id_list



In [145]:
nm = len(movie_id_list)
nu = len(sliced_user_dict)
#stores ratings
Y = np.zeros((nm, nu))
#binary valued indicator to check for if a user voted or not
R = np.zeros((nm,nu))
#stores ratings of users for each movie
Y_i = np.zeros(nu)
#binary valued indicator to check for if a user voted or not for each movie
R_i = np.zeros(nu)

for i in range(nm):
    found = False
    #iterate through movie list
    for j, (user,movies) in enumerate(sliced_user_dict.items()):
        #check if movie id is in the list of movie ids this user has rated
        for movie in movies:
            if movie[0] == movie_id_list[i]:
                #if the movie is in the list then store the ratings
                Y_i[j] = movie[1]
                R_i[j] = 1
                found = True
        if not found: #found is false
            #if the movie is not in their movie list then it is not rated by that user
            Y_i[j] = 0
            R_i[j] = 0
    Y[i] = Y_i
    R[i] = R_i
print(Y)
print(R)

        


    

        
        
    

[[0. 0. 0. ... 0. 4. 5.]
 [0. 0. 0. ... 0. 4. 5.]
 [0. 0. 0. ... 0. 4. 5.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 1. 1.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [155]:
#defining the input
X = df_movie["genres"].values
print(X)

['Adventure|Animation|Children|Comedy|Fantasy'
 'Adventure|Children|Fantasy' 'Comedy|Romance' ... 'Documentary' 'Comedy'
 'Documentary']


In [157]:
from sklearn.feature_extraction.text import CountVectorizer
#count vectorization
vectorizer = CountVectorizer()
x_bow = vectorizer.fit_transform(X)
x_enc = x_bow.toarray()
x_enc = x_enc.astype(float)
print(x_enc)

[[0. 1. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [148]:
m,n = x_enc.shape


In [149]:
def cofi_cost_func_v(X, W, b, Y, R, lambda_):
    """
    Returns the cost for the content-based filtering
    Vectorized for speed. Uses tensorflow operations to be compatible with custom training loop.
    Args:
      X (ndarray (num_movies,num_features)): matrix of item features
      W (ndarray (num_users,num_features)) : matrix of user parameters
      b (ndarray (1, num_users)            : vector of user parameters
      Y (ndarray (num_movies,num_users)    : matrix of user ratings of movies
      R (ndarray (num_movies,num_users)    : matrix, where R(i, j) = 1 if the i-th movies was rated by the j-th user
      lambda_ (float): regularization parameter
    Returns:
      J (float) : Cost
    """
    #make sure X and W have the same data type
    j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y)*R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    return J

In [158]:
import tensorflow as tf
from tensorflow import keras

# Set Initial Parameters (W, X), use tf.Variable to track these variables
tf.random.set_seed(1234) # for consistent results
print(nm)
print(nu)
print(n)
W = tf.Variable(tf.random.normal((nu,  n),dtype=tf.float64),  name='W')
print(W)
X = tf.Variable(tf.random.normal((nm, n),dtype=tf.float64),  name='X')
print(X.shape)
b = tf.Variable(tf.random.normal((1,nu),   dtype=tf.float64),  name='b')

# Instantiate an optimizer.
optimizer = keras.optimizers.Adam(learning_rate=1e-1)

9125
671
24
<tf.Variable 'W:0' shape=(671, 24) dtype=float64, numpy=
array([[-1.2650278 , -0.88266007,  0.45325746, ..., -0.5071518 ,
        -0.69534167,  1.78964465],
       [-0.54837295, -1.48310151,  0.38314998, ..., -0.96341192,
        -0.19556525, -0.49596018],
       [-0.15707949, -1.31670428,  0.53294877, ..., -0.25568902,
        -0.77758611,  0.93006297],
       ...,
       [ 0.41630287, -0.81200051,  1.71935755, ..., -1.84422598,
        -0.11612442, -0.52074994],
       [-0.57502626, -0.90119769,  0.4656603 , ..., -1.82202762,
        -0.53223653, -0.94507692],
       [-0.08472545,  0.0093382 , -0.7313654 , ..., -0.54756592,
         0.28952935,  0.48042513]])>
(9125, 24)


In [161]:
iterations = 200
lambda_ = 1
for iter in range(iterations):
    # Use TensorFlow’s GradientTape
    # to record the operations used to compute the cost 
    with tf.GradientTape() as tape:

        # Compute the cost (forward pass included in cost)
        cost_value = cofi_cost_func_v(x_enc, W, b, Y, R, lambda_)
        print(cost_value)
        #make sure cost value data type is compatible with the requirements of tape.gradient()

    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss
    grads = tape.gradient( cost_value, [X,W,b] )

    # Run one step of gradient descent by updating
    # the value of the variables to minimize the loss.
    optimizer.apply_gradients( zip(grads, [X,W,b]) )

    # Log periodically.
    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")

tf.Tensor(5683465.426079654, shape=(), dtype=float64)
[None, <tf.Tensor: id=76714, shape=(671, 24), dtype=float64, numpy=
array([[-2.25022872e+01, -1.92060290e+01,  1.91537194e-02, ...,
        -1.95035004e+01, -4.31508268e+00,  1.88593541e+00],
       [-7.51726360e+01, -7.11627189e+01, -3.17206005e+01, ...,
        -9.00642261e+01, -2.37802074e+01, -1.31606516e+01],
       [-8.03224018e+01, -4.95760742e+01, -6.64114970e+00, ...,
        -6.09353945e+01, -3.67578345e+01, -8.88423957e+00],
       ...,
       [-2.55233760e+03, -2.27993267e+03, -4.95870734e+02, ...,
        -3.53465859e+03, -4.76432160e+02, -3.60628950e+02],
       [-2.46492792e+03, -1.86672329e+03, -3.33788699e+02, ...,
        -2.96586292e+03, -4.74120156e+02, -3.52282394e+02],
       [-1.26239860e+03, -9.06389366e+02, -1.90647881e+02, ...,
        -1.43732341e+03, -3.87550793e+02, -1.91108910e+02]])>, <tf.Tensor: id=76710, shape=(1, 671), dtype=float64, numpy=
array([[-5.06279981e+01, -3.25233014e+02, -2.61378913e+02,


Training loss at iteration 0: 5683465.4
tf.Tensor(4941184.8460375955, shape=(), dtype=float64)
[None, <tf.Tensor: id=76841, shape=(671, 24), dtype=float64, numpy=
array([[-2.01022877e+01, -1.60059982e+01,  5.19213169e-01, ...,
        -1.69035011e+01, -3.91508284e+00,  1.68593605e+00],
       [-6.67726363e+01, -6.49627192e+01, -2.85206007e+01, ...,
        -8.12642264e+01, -2.15802075e+01, -1.21606516e+01],
       [-7.19224024e+01, -4.46760725e+01, -5.44114722e+00, ...,
        -5.53353948e+01, -3.44578345e+01, -7.98423968e+00],
       ...,
       [-2.32973760e+03, -2.09913266e+03, -4.38070733e+02, ...,
        -3.27585858e+03, -4.26732159e+02, -3.30828950e+02],
       [-2.22752791e+03, -1.67502328e+03, -2.74388699e+02, ...,
        -2.69156291e+03, -4.21520155e+02, -3.22082393e+02],
       [-1.02849860e+03, -7.16289364e+02, -1.23547881e+02, ...,
        -1.14462341e+03, -3.20750792e+02, -1.55508910e+02]])>, <tf.Tensor: id=76837, shape=(1, 671), dtype=float64, numpy=
array([[-4.4527955

tf.Tensor(4279787.004215646, shape=(), dtype=float64)
[None, <tf.Tensor: id=76950, shape=(671, 24), dtype=float64, numpy=
array([[-1.77282137e+01, -1.29574123e+01,  6.82107851e-01, ...,
        -1.43329376e+01, -3.51676236e+00,  1.48770358e+00],
       [-5.84103288e+01, -5.87878627e+01, -2.53328255e+01, ...,
        -7.25010435e+01, -1.93880152e+01, -1.11639253e+01],
       [-6.35840537e+01, -3.97891827e+01, -4.23397509e+00, ...,
        -4.97634546e+01, -3.21642933e+01, -7.08743071e+00],
       ...,
       [-2.10782050e+03, -1.91888094e+03, -3.80466510e+02, ...,
        -3.01785598e+03, -3.77202849e+02, -3.01120601e+02],
       [-1.99119760e+03, -1.48436236e+03, -2.15545041e+02, ...,
        -2.41856991e+03, -3.69175778e+02, -2.92019886e+02],
       [-7.98218065e+02, -5.29764360e+02, -5.91129523e+01, ...,
        -8.58000141e+02, -2.54615180e+02, -1.20392974e+02]])>, <tf.Tensor: id=76946, shape=(1, 671), dtype=float64, numpy=
array([[-3.88492374e+01, -2.68345240e+02, -2.20656991e+02,


tf.Tensor(3698139.327508841, shape=(), dtype=float64)
[None, <tf.Tensor: id=77059, shape=(671, 24), dtype=float64, numpy=
array([[-1.54118202e+01, -1.00558103e+01,  7.15423684e-01, ...,
        -1.18280277e+01, -3.12139314e+00,  1.29028107e+00],
       [-5.01188599e+01, -5.26597688e+01, -2.21675173e+01, ...,
        -6.38065605e+01, -1.72100828e+01, -1.01731395e+01],
       [-5.53956840e+01, -3.49849123e+01, -3.07358873e+00, ...,
        -4.42538034e+01, -2.98823686e+01, -6.19646367e+00],
       ...,
       [-1.88713820e+03, -1.73962024e+03, -3.23220331e+02, ...,
        -2.76129831e+03, -3.27985242e+02, -2.71577934e+02],
       [-1.75687388e+03, -1.29579676e+03, -1.57988874e+02, ...,
        -2.14804960e+03, -3.17319606e+02, -2.62222706e+02],
       [-5.77169818e+02, -3.51986114e+02, -1.35357944e+00, ...,
        -5.87502437e+02, -1.89873225e+02, -8.63704970e+01]])>, <tf.Tensor: id=77055, shape=(1, 671), dtype=float64, numpy=
array([[-3.34350280e+01, -2.40165453e+02, -2.00602219e+02,


tf.Tensor(3193570.3537206077, shape=(), dtype=float64)
[None, <tf.Tensor: id=77168, shape=(671, 24), dtype=float64, numpy=
array([[-1.32011766e+01, -7.37244408e+00,  6.79870919e-01, ...,
        -9.44372418e+00, -2.73045778e+00,  1.09360920e+00],
       [-4.19390119e+01, -4.66045824e+01, -1.90367868e+01, ...,
        -5.52195431e+01, -1.50539584e+01, -9.19137934e+00],
       [-4.74604430e+01, -3.03291425e+01, -2.00258557e+00, ...,
        -3.88448671e+01, -2.76178827e+01, -5.31442431e+00],
       ...,
       [-1.66832631e+03, -1.56186024e+03, -2.66523061e+02, ...,
        -2.50693465e+03, -2.79244586e+02, -2.42286161e+02],
       [-1.52569857e+03, -1.11072548e+03, -1.02800214e+02, ...,
        -1.88144547e+03, -2.66241683e+02, -2.32852053e+02],
       [-3.72415132e+02, -1.87913048e+02,  4.75259976e+01, ...,
        -3.45741686e+02, -1.27332993e+02, -5.39631068e+01]])>, <tf.Tensor: id=77164, shape=(1, 671), dtype=float64, numpy=
array([[-2.83531562e+01, -2.12308761e+02, -1.81005485e+02,

...

tf.Tensor(282600.771036292, shape=(), dtype=float64)
[None, <tf.Tensor: id=97987, shape=(671, 24), dtype=float64, numpy=
array([[ 3.89678814e-04,  5.81680475e-03, -1.52909149e-04, ...,
         8.46625679e-03,  5.48475929e-04,  7.56005902e-04],
       [ 2.39865260e-01,  2.34912366e-02,  8.60197088e-02, ...,
         1.70484933e-01,  5.59603772e-03,  1.75986643e-02],
       [ 2.70105448e-01,  4.54078931e-02,  1.15051641e-02, ...,
         1.43977494e-01,  4.68575786e-02,  1.44167634e-02],
       ...,
       [ 7.41751518e+00,  4.46339873e+00,  6.88679571e-01, ...,
         8.21783805e+00,  8.26743900e-01,  1.10269123e+00],
       [ 2.95964558e+00,  1.47571595e+00,  1.41653682e-01, ...,
         3.01949793e+00,  3.15769426e-01,  4.80857854e-01],
       [ 4.12795310e-02,  1.99002340e-02,  1.65944446e-02, ...,
         6.44967767e-02,  3.10028292e-02,  1.16017591e-02]])>, <tf.Tensor: id=97983, shape=(1, 671), dtype=float64, numpy=
array([[-3.29716675e-02, -2.03640002e+00, -2.57878815e+00,
 

tf.Tensor(282371.192374868, shape=(), dtype=float64)
[None, <tf.Tensor: id=98096, shape=(671, 24), dtype=float64, numpy=
array([[ 4.13191576e-04,  5.60264471e-03, -1.48558875e-04, ...,
         8.09012802e-03,  5.36232642e-04,  7.15021678e-04],
       [ 2.37463391e-01,  2.35823240e-02,  8.49172289e-02, ...,
         1.69135641e-01,  5.67249261e-03,  1.74586701e-02],
       [ 2.66629456e-01,  4.44140509e-02,  1.14545312e-02, ...,
         1.42098288e-01,  4.61675517e-02,  1.42096361e-02],
       ...,
       [ 7.32241094e+00,  4.40490883e+00,  6.81143152e-01, ...,
         8.11030619e+00,  8.16130989e-01,  1.08831407e+00],
       [ 2.88619885e+00,  1.44086859e+00,  1.36485860e-01, ...,
         2.94283111e+00,  3.05370175e-01,  4.68484409e-01],
       [ 4.01567995e-02,  1.59398947e-02,  1.16147979e-02, ...,
         5.85959724e-02,  2.69149021e-02,  1.12031875e-02]])>, <tf.Tensor: id=98092, shape=(1, 671), dtype=float64, numpy=
array([[-3.14613872e-02, -2.00672267e+00, -2.55058371e+00,
 

tf.Tensor(282145.5005050847, shape=(), dtype=float64)
[None, <tf.Tensor: id=98205, shape=(671, 24), dtype=float64, numpy=
array([[ 4.25295378e-04,  5.38146504e-03, -1.30258471e-04, ...,
         7.71279685e-03,  5.18389937e-04,  6.61820824e-04],
       [ 2.34976196e-01,  2.36141226e-02,  8.38201139e-02, ...,
         1.67671374e-01,  5.72501219e-03,  1.73107875e-02],
       [ 2.63211095e-01,  4.34475244e-02,  1.13960178e-02, ...,
         1.40281317e-01,  4.55000874e-02,  1.40068248e-02],
       ...,
       [ 7.22388304e+00,  4.34396612e+00,  6.72642895e-01, ...,
         7.99933750e+00,  8.04702111e-01,  1.07354559e+00],
       [ 2.81837377e+00,  1.41043021e+00,  1.33038329e-01, ...,
         2.87223395e+00,  2.96110921e-01,  4.56947017e-01],
       [ 3.74941545e-02,  1.11539935e-02,  6.51708231e-03, ...,
         5.09049013e-02,  2.23093131e-02,  1.06238335e-02]])>, <tf.Tensor: id=98201, shape=(1, 671), dtype=float64, numpy=
array([[-3.00543855e-02, -1.97767222e+00, -2.52250326e+00,


        -3.98414839e+01, -8.66838914e-01]])>]
tf.Tensor(281923.6347868902, shape=(), dtype=float64)
[None, <tf.Tensor: id=98314, shape=(671, 24), dtype=float64, numpy=
array([[ 4.27769430e-04,  5.15636410e-03, -1.01706308e-04, ...,
         7.33784098e-03,  4.95294603e-04,  5.99796095e-04],
       [ 2.32371662e-01,  2.35699214e-02,  8.27222185e-02, ...,
         1.66060973e-01,  5.74736603e-03,  1.71520984e-02],
       [ 2.59883455e-01,  4.25291893e-02,  1.13319781e-02, ...,
         1.38537615e-01,  4.48574643e-02,  1.38103340e-02],
       ...,
       [ 7.12276858e+00,  4.28117360e+00,  6.63359285e-01, ...,
         7.88593170e+00,  7.92644542e-01,  1.05849655e+00],
       [ 2.75606889e+00,  1.38421252e+00,  1.31197152e-01, ...,
         2.80763811e+00,  2.88005236e-01,  4.46234432e-01],
       [ 3.38539195e-02,  6.18214323e-03,  1.70726756e-03, ...,
         4.24967931e-02,  1.75053003e-02,  9.95291849e-03]])>, <tf.Tensor: id=98310, shape=(1, 671), dtype=float64, numpy=
array([[-2.87

tf.Tensor(281705.53540458565, shape=(), dtype=float64)
[None, <tf.Tensor: id=98423, shape=(671, 24), dtype=float64, numpy=
array([[ 4.23317871e-04,  4.93156398e-03, -6.68653554e-05, ...,
         6.96972558e-03,  4.67552017e-04,  5.32836997e-04],
       [ 2.29629991e-01,  2.34397255e-02,  8.16196648e-02, ...,
         1.64285622e-01,  5.73584891e-03,  1.69807426e-02],
       [ 2.56671620e-01,  4.16751888e-02,  1.12643348e-02, ...,
         1.36874123e-01,  4.42408827e-02,  1.36216372e-02],
       ...,
       [ 7.01997530e+00,  4.21719336e+00,  6.53493255e-01, ...,
         7.77116344e+00,  7.80162763e-01,  1.04328681e+00],
       [ 2.69886754e+00,  1.36175966e+00,  1.30751104e-01, ...,
         2.74860633e+00,  2.80988340e-01,  4.36294353e-01],
       [ 2.99529386e-02,  1.73345846e-03, -2.43643057e-03, ...,
         3.45251940e-02,  1.28183978e-02,  9.28676650e-03]])>, <tf.Tensor: id=98419, shape=(1, 671), dtype=float64, numpy=
array([[-2.75104538e-02, -1.92174634e+00, -2.46653274e+00,

In [162]:
# Make a prediction using trained weights and biases
p = np.matmul(X.numpy(), np.transpose(W.numpy())) + b.numpy()
#Note: the rows represent each movie and the columns represent each user

print(p)


[[4.78301079 4.90449074 3.15575267 ... 2.06929674 4.97340879 4.20406771]
 [3.78884668 3.20290074 3.15343827 ... 5.51956631 1.6120456  3.79335014]
 [3.40355575 7.99690693 1.65664662 ... 1.59595368 5.21211661 4.17754716]
 ...
 [4.0837605  1.15774503 2.11286633 ... 2.05226433 3.6317934  3.42834999]
 [4.51080275 5.1448831  1.31747166 ... 3.52207191 3.39738848 3.68237086]
 [2.28708709 4.07145859 1.15706144 ... 2.07360663 4.03763743 3.66753127]]


In [163]:
#to pick a movie a user has not rated would be movie 1 and user 1 and based on the prediction it is most likely user 1
#will like movie 1
#Note: To find movies similar to each other, the value of the input features must be similar i.e to find a movie K similar
#to a movie i, Vector(X_i) and Vector(X_k) that are closest

print(X)

<tf.Variable 'X:0' shape=(9125, 24) dtype=float64, numpy=
array([[ 0.92169644,  0.40301904, -0.67974247, ..., -0.47733292,
         0.30809066,  0.07542342],
       [-0.45453283,  0.73742837, -0.23854552, ..., -0.74528388,
         1.38783616,  0.54653446],
       [-0.29656788,  0.40229388, -0.46436498, ..., -1.20075887,
         0.72424704,  0.10177682],
       ...,
       [ 0.88564007, -1.14287594,  1.60203391, ...,  1.02978707,
         1.24387193,  0.31709962],
       [-0.5768467 ,  0.29985087, -1.15710717, ..., -1.07555446,
        -0.61480494,  1.54751057],
       [-0.46585549,  0.92920776, -0.59434025, ...,  0.99183869,
         0.71474962,  0.59501854]])>
